# Подключаем пакеты и определяем функции

In [2]:
# Для работы с данными
import pandas as pd
import numpy as np
import wfdb
import ast
from utils import utils
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt   # plotting
#import seaborn as sns   # plotting heatmap

# Для работы с моделями
import tensorflow as tf
from tensorflow import keras
from keras import layers

# Для метрик
from keras import backend as K
from keras.metrics import AUC, Recall, Precision, Accuracy, TruePositives, TrueNegatives, FalsePositives, FalseNegatives
from sklearn.metrics import fbeta_score, precision_score, recall_score, accuracy_score, roc_auc_score

# Функции
# Компиляция и обучение модели
def compile_fit(model, X_train, y_train, X_val, y_val, early_stopping, model_checkpoint):
  model.compile(loss = keras.losses.CategoricalCrossentropy(),
                optimizer=tf.optimizers.Adam(),
                metrics=['accuracy'])

  history = model.fit(X_train, y_train, epochs = 30, validation_data = (X_val, y_val), callbacks=[model_checkpoint, early_stopping])
  return history

tf.random.set_seed(42)
%matplotlib inline

# Скачиваем ICBEB с использованием кода и обработки авторов исследуемой статьи (обработанные данные, т.е. проведена нормализация и категоризация)

In [2]:
sampling_frequency=100
datafolder='data/ICBEB/'
task='diagnostic'
#task='superdiagnostic'
#task = 'subdiagnostic'
#task = 'rhythm'
#task = 'form'
outputfolder='output/'

# Load ICBEB data
data, raw_labels = utils.load_dataset(datafolder, sampling_frequency)
# Preprocess label data
labels = utils.compute_label_aggregations(raw_labels, datafolder, task)
# Select relevant data and convert to one-hot
data, labels, Y, _ = utils.select_data(data, labels, task, min_samples=0, outputfolder=outputfolder)

# 1-9 for training 
X_train = data[labels.strat_fold < 10]
y_train = Y[labels.strat_fold < 10]
# 10 for validation
X_val = data[labels.strat_fold == 10]
y_val = Y[labels.strat_fold == 10]

# Стандартизация 3D данных c применением StandardScaler.
# Сначала изменяется форма данных а затем применяется нормализация. После этого требуется вернуть их прежнюю форму 
def standard_scaler(X_train, X_val): 
  scaler = StandardScaler()
  # Train
  num_instances, num_time_steps, num_features = X_train.shape
  X_train = np.reshape(X_train, newshape=(-1, num_features))
  X_train = scaler.fit_transform(X_train)
  X_train = np.reshape(X_train, newshape=(num_instances, num_time_steps, num_features))
  # Valid
  num_instances, num_time_steps, num_features = X_val.shape
  X_val = np.reshape(X_val, newshape=(-1, num_features))
  X_val = scaler.fit_transform(X_val)
  X_val = np.reshape(X_val, newshape=(num_instances, num_time_steps, num_features))
  return X_train, X_val

X_train.shape, y_train.shape, X_val.shape, y_val.shape

((3337, 1000, 12), (3337, 4), (378, 1000, 12), (378, 4))

##### task = 'diagnostic'

In [2]:
# # Выделим уникальные значения y_train в столбце и посчитаем их количество
# unique, counts = np.unique(y_train, return_counts = True, axis = 0) 
# print(unique, '\n', 'len(unique): ', len(unique))
# print(counts, '\n', 'len(counts): ', len(counts))

In [3]:
# Стандартизация данных
X_train, X_val = standard_scaler(X_train, X_val)

In [16]:
# Сохранение наборов данных в файлы .npy для дальнейшего использования в Google Colab
np.save('X_train_ICBEB_diag', X_train)
np.save('X_val_ICBEB_diag', X_val)
np.save('y_train_ICBEB_diag', y_train)
np.save('y_val_ICBEB_diag', y_val)

##### task = 'superdiagnostic'

In [4]:
# # Выделим уникальные значения y_train в столбце и посчитаем их количество
# unique, counts = np.unique(y_train, return_counts = True, axis = 0) 
# print(unique, '\n', 'len(unique): ', len(unique))
# print(counts, '\n', 'len(counts): ', len(counts))

In [3]:
# Стандартизация данных
X_train, X_val = standard_scaler(X_train, X_val)

# Сохранение наборов данных в файлы .npy для дальнейшего использования в Google Colab
np.save('X_train_ICBEB_superdiag', X_train)
np.save('X_val_ICBEB_superdiag', X_val)
np.save('y_train_ICBEB_superdiag', y_train)
np.save('y_val_ICBEB_superdiag', y_val)

##### task = 'subdiagnostic'

In [3]:
# # Выделим уникальные значения y_train в столбце и посчитаем их количество
# unique, counts = np.unique(y_train, return_counts = True, axis = 0) 
# print(unique, '\n', 'len(unique): ', len(unique))
# print(counts, '\n', 'len(counts): ', len(counts))

In [3]:
# Стандартизация данных
X_train, X_val = standard_scaler(X_train, X_val)

# Сохранение наборов данных в файлы .npy для дальнейшего использования в Google Colab
np.save('X_train_ICBEB_subdiag', X_train)
np.save('X_val_ICBEB_subdiag', X_val)
np.save('y_train_ICBEB_subdiag', y_train)
np.save('y_val_ICBEB_subdiag', y_val)

##### task = 'rhythm'

In [5]:
# # Выделим уникальные значения y_train в столбце и посчитаем их количество
# unique, counts = np.unique(y_train, return_counts = True, axis = 0) 
# print(unique, '\n', 'len(unique): ', len(unique))
# print(counts, '\n', 'len(counts): ', len(counts))

In [3]:
# Стандартизация данных
X_train, X_val = standard_scaler(X_train, X_val)

# Сохранение наборов данных в файлы .npy для дальнейшего использования в Google Colab
np.save('X_train_ICBEB_rhythm', X_train)
np.save('X_val_ICBEB_rhythm', X_val)
np.save('y_train_ICBEB_rhythm', y_train)
np.save('y_val_ICBEB_rhythm', y_val)

##### task = 'form'

In [7]:
# # Выделим уникальные значения y_train в столбце и посчитаем их количество
# unique, counts = np.unique(y_train, return_counts = True, axis = 0) 
# print(unique, '\n', 'len(unique): ', len(unique))
# print(counts, '\n', 'len(counts): ', len(counts))

In [3]:
# Стандартизация данных
X_train, X_val = standard_scaler(X_train, X_val)

# Сохранение наборов данных в файлы .npy для дальнейшего использования в Google Colab
np.save('X_train_ICBEB_form', X_train)
np.save('X_val_ICBEB_form', X_val)
np.save('y_train_ICBEB_form', y_train)
np.save('y_val_ICBEB_form', y_val)

# Скачиваем PTB-XL с использованием кода и обработки авторов исследуемой статьи (обработанные данные, т.е. проведена нормализация и категоризация)

In [2]:
sampling_frequency=100
datafolder='data/ptbxl/'
#task='diagnostic'
#task='superdiagnostic'
#task = 'subdiagnostic'
#task = 'rhythm'
task = 'form'
outputfolder='output/'

# Load PTB-XL data
data, raw_labels = utils.load_dataset(datafolder, sampling_frequency)
# Preprocess label data
labels = utils.compute_label_aggregations(raw_labels, datafolder, task)
# Select relevant data and convert to one-hot
data, labels, Y, _ = utils.select_data(data, labels, task, min_samples=0, outputfolder=outputfolder)

# 1-9 for training 
X_train = data[labels.strat_fold < 10]
y_train = Y[labels.strat_fold < 10]
# 10 for validation
X_val = data[labels.strat_fold == 10]
y_val = Y[labels.strat_fold == 10]

# Стандартизация 3D данных c применением StandardScaler.
# Сначала изменяется форма данных а затем применяется нормализация. После этого требуется вернуть их прежнюю форму 
def standard_scaler(X_train, X_val): 
  scaler = StandardScaler()
  # Train
  num_instances, num_time_steps, num_features = X_train.shape
  X_train = np.reshape(X_train, newshape=(-1, num_features))
  X_train = scaler.fit_transform(X_train)
  X_train = np.reshape(X_train, newshape=(num_instances, num_time_steps, num_features))
  # Valid
  num_instances, num_time_steps, num_features = X_val.shape
  X_val = np.reshape(X_val, newshape=(-1, num_features))
  X_val = scaler.fit_transform(X_val)
  X_val = np.reshape(X_val, newshape=(num_instances, num_time_steps, num_features))
  return X_train, X_val

X_train.shape, y_train.shape, X_val.shape, y_val.shape

((8106, 1000, 12), (8106, 19), (882, 1000, 12), (882, 19))

##### task = 'diagnostic'

In [8]:
# # Выделим уникальные значения y_train в столбце и посчитаем их количество
# unique, counts = np.unique(y_train, return_counts = True, axis = 0) 
# print(unique, '\n', 'len(unique): ', len(unique))
# print(counts, '\n', 'len(counts): ', len(counts))

In [3]:
# Стандартизация данных
X_train, X_val = standard_scaler(X_train, X_val)

# Сохранение наборов данных в файлы .npy для дальнейшего использования в Google Colab
np.save('X_train_ptbxl_diag', X_train)
np.save('X_val_ptbxl_diag', X_val)
np.save('y_train_ptbxl_diag', y_train)
np.save('y_val_ptbxl_diag', y_val)

##### task = 'superdiagnostic'

In [9]:
# # Выделим уникальные значения y_train в столбце и посчитаем их количество
# unique, counts = np.unique(y_train, return_counts = True, axis = 0) 
# print(unique, '\n', 'len(unique): ', len(unique))
# print(counts, '\n', 'len(counts): ', len(counts))

In [3]:
# Стандартизация данных
X_train, X_val = standard_scaler(X_train, X_val)

# Сохранение наборов данных в файлы .npy для дальнейшего использования в Google Colab
np.save('X_train_ptbxl_superdiag', X_train)
np.save('X_val_ptbxl_superdiag', X_val)
np.save('y_train_ptbxl_superdiag', y_train)
np.save('y_val_ptbxl_superdiag', y_val)

##### task = 'subdiagnostic'

In [10]:
# # Выделим уникальные значения y_train в столбце и посчитаем их количество
# unique, counts = np.unique(y_train, return_counts = True, axis = 0) 
# print(unique, '\n', 'len(unique): ', len(unique))
# print(counts, '\n', 'len(counts): ', len(counts))

In [3]:
# Стандартизация данных
X_train, X_val = standard_scaler(X_train, X_val)

# Сохранение наборов данных в файлы .npy для дальнейшего использования в Google Colab
np.save('X_train_ptbxl_subdiag', X_train)
np.save('X_val_ptbxl_subdiag', X_val)
np.save('y_train_ptbxl_subdiag', y_train)
np.save('y_val_ptbxl_subdiag', y_val)

##### task = 'rhythm'

In [11]:
# # Выделим уникальные значения y_train в столбце и посчитаем их количество
# unique, counts = np.unique(y_train, return_counts = True, axis = 0) 
# print(unique, '\n', 'len(unique): ', len(unique))
# print(counts, '\n', 'len(counts): ', len(counts))

In [3]:
# Стандартизация данных
X_train, X_val = standard_scaler(X_train, X_val)

# Сохранение наборов данных в файлы .npy для дальнейшего использования в Google Colab
np.save('X_train_ptbxl_rhythm', X_train)
np.save('X_val_ptbxl_rhythm', X_val)
np.save('y_train_ptbxl_rhythm', y_train)
np.save('y_val_ptbxl_rhythm', y_val)

##### task = 'form'

In [12]:
# # Выделим уникальные значения y_train в столбце и посчитаем их количество
# unique, counts = np.unique(y_train, return_counts = True, axis = 0) 
# print(unique, '\n', 'len(unique): ', len(unique))
# print(counts, '\n', 'len(counts): ', len(counts))

In [3]:
# Стандартизация данных
X_train, X_val = standard_scaler(X_train, X_val)

# Сохранение наборов данных в файлы .npy для дальнейшего использования в Google Colab
np.save('X_train_ptbxl_form', X_train)
np.save('X_val_ptbxl_form', X_val)
np.save('y_train_ptbxl_form', y_train)
np.save('y_val_ptbxl_form', y_val)

# Скачиваем ICBEB через форму PTB-XL

In [8]:
def load_raw_data(df, sampling_rate, path):
    if sampling_rate == 100:
        data = [wfdb.rdsamp(path+f) for f in df.filename]
    data = [signal for signal, meta in data]
    for index, value in enumerate(data):
      if len(value) < 1000:
        data.remove(value)
        df = df.drop(index = index)
    refLen = 1000 # reference - эталон
    for index, value in enumerate(data):
      if len(value) > 1000:
        data[index] = data[index][:refLen]
    data = np.array(data)
    return data, df

path = 'data/ICBEB/'
sampling_rate=100

# load and convert annotation data
Y = pd.read_csv(path+'icbeb_database.csv', index_col='ecg_id')
Y.scp_codes = Y.scp_codes.apply(lambda x: ast.literal_eval(x))

# Load raw signal data
X, Y = load_raw_data(Y, sampling_rate, path)

# Load scp_statements.csv for diagnostic aggregation
agg_df = pd.read_csv(path+'scp_statements.csv', index_col=0)
agg_df = agg_df[agg_df.form == 1]

def aggregate_diagnostic(y_dic):
    tmp = []
    for key in y_dic.keys():
        if key in agg_df.index:
            tmp.append(agg_df.loc[key].diagnostic_class)
    return list(set(tmp))

# Apply diagnostic superclass
Y['diagnostic_superclass'] = Y.scp_codes.apply(aggregate_diagnostic)

# # Split data into train and test
test_fold = 10
# Train
X_train = X[np.where(Y.strat_fold != test_fold)]
y_train = Y[(Y.strat_fold != test_fold)].diagnostic_superclass
# Test
X_test = X[np.where(Y.strat_fold == test_fold)]
y_test = Y[Y.strat_fold == test_fold].diagnostic_superclass

C:\Users\manuk\AppData\Local\Temp\ipykernel_7528\110442798.py:7: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  data.remove(value)
C:\Users\manuk\AppData\Local\Temp\ipykernel_7528\110442798.py:7: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  data.remove(value)
C:\Users\manuk\AppData\Local\Temp\ipykernel_7528\110442798.py:7: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  data.remove(value)
C:\Users\manuk\AppData\Local\Temp\ipykernel_7528\110442798.py:7: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  data.remove(value)
C:\Users\manuk\AppData\Local\Temp\ipykernel_7528\110442798.py:7: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  data.remove(value)
C:\Users\manuk\AppData\Local\Temp\ipykernel_7528\110442798.py:7: DeprecationWarning: elementwise comparison failed;

In [9]:
X_train.shape, y_train.shape

((6181, 1000, 12), (6181,))

In [10]:
X_train[0]

array([[ 0.02822609,  0.00672717, -0.02150135, ..., -0.11200486,
        -0.59595824, -0.01558333],
       [ 0.05623843,  0.02373605, -0.03248586, ..., -0.09599664,
        -0.55091871,  0.03543248],
       [ 0.06525131,  0.05676969, -0.00847507, ..., -0.05695029,
        -0.48688198,  0.10148025],
       ...,
       [ 0.19612203, -0.16015955, -0.35627225, ...,  0.38222251,
         0.2396717 , -0.25792291],
       [-0.01297901, -0.48225593, -0.46927696, ..., -0.44072623,
        -0.56097927, -0.85331045],
       [-0.34466227, -0.59409148, -0.24942707, ..., -0.98916564,
        -1.00918153, -1.11767564]])

In [12]:
y_train

ecg_id
1          []
2          []
3          []
5          []
6          []
        ...  
6873       []
6874    [nan]
6875       []
6876       []
6877       []
Name: diagnostic_superclass, Length: 6181, dtype: object

In [13]:
Y

,filename,validation,age,sex,scp_codes,patient_id,quality,strat_fold,diagnostic_superclass
ecg_id,,,,,,,,,
1,records100/1,False,74.0,1,{'CRBBB': 100},1,0,7,[]
2,records100/2,False,49.0,0,{'NORM': 100},2,0,7,[]
3,records100/3,False,81.0,0,{'AFIB': 100},3,0,3,[]
4,records100/4,False,45.0,1,{'AFIB': 100},4,0,10,[]
5,records100/5,False,53.0,1,{'VPC': 100},5,0,7,[]
...,...,...,...,...,...,...,...,...,...
6873,records100/6873,False,80.0,1,{'1AVB': 100},6873,0,5,[]
6874,records100/6874,False,62.0,0,{'STD_': 100},6874,0,1,[nan]
6875,records100/6875,False,78.0,1,{'CLBBB': 100},6875,0,5,[]


In [15]:
# Создать Ndarray Numpy копию Series Pandas.
y_trainNp = y_train.to_numpy()
print(type(y_trainNp))
print(y_trainNp.size)

<class 'numpy.ndarray'>
6181


In [16]:
# Выделим уникальные значения y_train_np и посчитаем их количество.
unique, counts = np.unique(y_trainNp, return_counts = True)
print(unique, '\n', "unique.size: ", unique.size)
print(counts)

[list([]) list([nan])] 
 unique.size:  2
[4658 1523]


In [5]:
import pandas as pd
table = pd.read_csv('table_res_finetuning.csv', index_col=0)
table

,AUC,F2,G2
lstm_ICBEB_form,0.792765,0.498645,0.281553
lstm_bidir_ICBEB_form,0.895261,0.747274,0.512097
lstm_ptbxl_form,0.804124,0.041219,0.000000
lstm_bidir_ptbxl_form,0.802981,0.041219,0.000000
lstm_finetuning_form,0.691744,0.279402,0.000000
